In [ ]:
!pip install --user --upgrade --editable gym-env

# Restart Kernel

In [ ]:
import gym
from gym import spaces
import numpy as np
from numpy import random as rand
import random
import math
import matplotlib.pyplot as plt
import sys

In [ ]:
env = gym.make('gym_env:foraging-v102') #rich patch poor patch environment

In [ ]:
noEpisodes=1000
mp=100
mh=500
noEnvs=10
g=0.7

In [ ]:
def decay(initial, final, stepstoDecay,noEpisodes, type):
    x=np.zeros(noEpisodes)
    
    a=stepstoDecay
    k = np.log(initial/final)/(a-1)
    for e in range(a):
        if type=="linear":
            x[e]=(initial-(e)*(initial-final)/(a-1))
        else:
            x[e]=(initial*np.exp(-(e)*k))
    for e in range(a,noEpisodes):
        x[e]=final     
    return x
   

In [ ]:
alpha = decay(1,0.1,noEpisodes,noEpisodes,"exp")/100000

epsilon = decay(1,0.0001,600,noEpisodes,"exp")

In [ ]:
def actionSelect(p,h,Q,epsilon):
    if random.random()>epsilon:
        a = np.argmax(Q[p][h])
        pos = []
        for i in range(len(Q[p][h])):
            if(Q[p][h][i]==Q[p][h][a]):
                pos.append(i)
        a = random.choice(pos)
    else:
        a = random.randint(0,1)
    return a


In [ ]:
def q_learning(env,mp,mh,gamma,alpha0,epsilon0,noEpisodes):
    #initialisation
    P=np.zeros(noEpisodes)
    rho=0
    Q=np.zeros((mp,mh,env.action_space.n))
    Qs=np.zeros((noEpisodes,mp,mh,env.action_space.n))
    #Algorithm
    R=np.zeros((noEpisodes))
    for e in range (noEpisodes):
        h,done=env.reset()
        p=0
        alpha=alpha0[e]
        epsilon=epsilon0[e]
        # print(t)
        r=0
        while not done:
            a=actionSelect(p,h,Q,epsilon)
            h_next,reward,done = env.step(a)
            if a==0 :
            # if h_next==0:
                p_next=p+1
            else:
                p_next=p
            td_target=reward
            r+=reward
            if not done:
                td_target=td_target+gamma*np.amax(Q[p_next][h_next])
            td_error = td_target - Q[p][h][a]-rho
            Q[p][h][a]=Q[p][h][a]+alpha*td_error
            p,h=p_next,h_next
        P[e]=rho
        rho=(e*rho+(r/240))/(e+1)
            
        R[e]=r #bookkeeping for rewards
        Qs[e]=Q
    # V,pi=np.amax(Q,axis=2),np.argmax(Q,axis=2)
    return R
   

In [ ]:
R1=[]
for i in range(noEnvs):
    # print(i)

    env1 = gym.make('gym_env:foraging-v102', interval_time = 10, decision_time="V")

    env1.seed(i+50)

    random.seed(i+50)
    r1=q_learning(env1,mp,mh,g,alpha,epsilon,noEpisodes)

    R1.append(r1)

R1=np.array(R1)

A1=np.average(R1,axis=0)

F=np.zeros(noEpisodes)

for e in range(noEpisodes):
    F[e]=np.mean(A1[e-99:e+1])

plt.plot(F, color = 'blue',label="Uniform (0.6-1.4) sec")
plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.title("Travel Time =10 seconds")
plt.savefig('2D R .pdf')
plt.show()
plt.close()

'''
    Here we plot for decision time = 3 seconds


'''
U1=[]

for i in range(noEnvs):
    # print(i)
    env1 = gym.make('gym_env:foraging-v102', interval_time = 3, decision_time="V")
    
    env1.seed(i+50)

    random.seed(i+50)
    r1=q_learning(env1,mp,mh,g,alpha,epsilon,noEpisodes)

    U1.append(r1)

U1=np.array(U1)

A1=np.average(U1,axis=0)

for e in range(noEpisodes):
    F[e]=np.mean(A1[e-99:e+1])

plt.plot(F, color = 'blue',label="Uniform (0.6-1.4) sec")


plt.xlabel("Episodes")
plt.ylabel("Rewards")
plt.title("Travel Time = 3 seconds")
plt.savefig('2D R tt=3.pdf')
plt.show()
plt.close()

